In [ ]:
import hpelm
from sklearn.metrics import precision_recall_fscore_support,ConfusionMatrixDisplay
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPool2D,BatchNormalization
from keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam,SGD

from keras.layers import Dense, Activation, Flatten,Dropout
import numpy as np
import pandas as pd
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Layout, Figure, Marker
import tensorflow as tf
from matplotlib import pyplot as plt
import time

import os
import cv2
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn import metrics
import seaborn as sns
NUM_CLASS = 2
CNN_EPOCH = 10
ELM_HIDDEN_NEURONS = 250

In [ ]:
def load_images_and_labels(data_path, cates): 
  X = []
  y = []
  i = 0
  for index, cate in enumerate(cates): 
    for img_name in os.listdir(data_path + cate + '/'):
      i = i +1
      #print(i)
      img = cv2.imread(data_path + cate + '/' + img_name)
      if img is not None: 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_array = Image.fromarray(img, 'RGB')
        
        img_rs = img_array.resize((32,32))
       
        img_rs = np.array(img_rs)
      
        X.append(img_rs)
        y.append(index)
  return X, y
def preprocess_data(X, y):
  # convert X from list to array
  X = np.array(X)
  
  # convert integer values of X into floats
  X = X.astype(np.float32)

  # normalization 
  X = X/255.0
  
  # one-hot encoding the labels 
  y = to_categorical(np.array(y))
    
  return X, y
def cnn_generate():
    cnn_model = Sequential()
    cnn_model.add(Conv2D(filters=32,kernel_size=3, activation='relu',padding='same',input_shape=(32,32,3)))    
    cnn_model.add(BatchNormalization()) 
    cnn_model.add(Conv2D(filters=32,kernel_size=3, activation='relu',padding='same'))    
    cnn_model.add(BatchNormalization())  
    cnn_model.add(Conv2D(filters=32,kernel_size=5, activation='relu',padding='same',strides=(2,2)))    
    cnn_model.add(BatchNormalization())    
    cnn_model.add(Dropout(0.5))

    cnn_model.add(Conv2D(filters=64,kernel_size=3, activation='relu',padding='same'))    
    cnn_model.add(BatchNormalization()) 
    cnn_model.add(Conv2D(filters=64,kernel_size=3, activation='relu',padding='same'))    
    cnn_model.add(BatchNormalization()) 
    cnn_model.add(Conv2D(filters=64,kernel_size=5, activation='relu',padding='same',strides=(2,2)))    
    cnn_model.add(BatchNormalization())     
    cnn_model.add(Dropout(0.5))
    
    
    cnn_model.add(Flatten())

    cnn_model.add(Dense(256))
    cnn_model.add(Activation('relu'))
    cnn_model.add(Dropout(0.5))
    
    cnn_model.add(Dense(2, activation='softmax'))
    opt= Adam(learning_rate=0.001)
    #opt = SGD(lr = 0.001,decay=0.9,momentum=0.9)
    
    cnn_model.summary()
    cnn_model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])    

    #history = cnn_model.fit(data_train_2d, target_train_oh, batch_size=20, epochs=CNN_EPOCH, verbose=1, validation_split=0.2)

    return cnn_model

In [ ]:


def train_model(model, X_train, y_train, X_test, y_test, epochs, batch_size):
  # Data generator
  datagen = ImageDataGenerator(rotation_range = 5, width_shift_range = 0.1, height_shift_range = 0.1, 
                               horizontal_flip = True)
  # iteration on the training set
  it_train = datagen.flow(X_train, y_train, batch_size = batch_size)
  # path to save checkpoint 
  path_cp = os.getcwd() + '/' + 'weights_.hdf5'
  checkpoint_ = ModelCheckpoint(path_cp, monitor = 'loss', save_best_only = True, mode = 'auto')
  steps = X_train.shape[0]//batch_size
  # Fitting the model
  history = model.fit(it_train, epochs = epochs, steps_per_epoch = steps, 
                                validation_data = (X_test, y_test), verbose = 1, 
                                callbacks = checkpoint_)
  # Evaluating the model
  _, acc = model.evaluate(X_test, y_test, verbose = 1)
  print('%.3f' % (acc * 100.0))
  p1=plt
  p1.plot(history.history['accuracy'])
  p1.plot(history.history['val_accuracy'])
  p1.title('model accuracy')
  p1.ylabel('accuracy')
  p1.xlabel('epoch')
  p1.legend(['train', 'test'], loc='upper left')
  p1.show()


  p2=plt
  #training loss vs validation loss
  p2.plot(history.history['loss'])
  p2.plot(history.history['val_loss'])
  p2.title('model loss')
  p2.ylabel('loss')
  p2.xlabel('epoch')
  p2.legend(['train', 'test'], loc='upper left')
  p2.show()
  return history, acc,model

In [ ]:
train_path = r"E:\Traffic congestion classification\Dataset\CNN-IELM\train/"
test_path=r"E:\Traffic congestion classification\Dataset\CNN-IELM\test/"
cates = ['congested','uncongested']
#loading
X_train, y_train = load_images_and_labels(train_path, cates)
X_test, y_test = load_images_and_labels(test_path, cates)

#loading
X_train, y_train = load_images_and_labels(train_path, cates)
X_test, y_test = load_images_and_labels(test_path, cates)

#preprocessing
(X_train, y_train) = preprocess_data(X_train, y_train)
(X_test, y_test) = preprocess_data(X_test, y_test)
